# Seeds

In [1]:
# Cell 0 — Seeds (minimal)
import os, random, numpy as np, torch
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


# chargement et pretraitement des donnees

Imports & schéma de colonnes

In [2]:
# =========================
# Cell 1 — Imports & colonnes
# =========================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

# Schéma des colonnes du WDBC (Breast Cancer Wisconsin)
columns = [
    "id", "diagnosis",
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean",
    "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se",
    "compactness_se", "concavity_se", "concave_points_se", "symmetry_se", "fractal_dimension_se",
    "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
    "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"
]


Chargement + préparation X/y

In [3]:
# =========================
# Cell 2 — Chargement CSV & X/y
# =========================
df = pd.read_csv("wdbc.data", header=None, names=columns)

# Séparer X (features) et y (target binaire)
X = df.drop(['id', 'diagnosis'], axis=1)
y = df['diagnosis'].map({'B': 0, 'M': 1})  # 0 = Bénin, 1 = Malin

print("Dimensions complètes :", X.shape)
print("Répartition classes :", y.value_counts().to_dict())


Dimensions complètes : (569, 30)
Répartition classes : {0: 357, 1: 212}


Split STRATIFIÉ + Scaling fit-on-train

In [4]:
# =========================
# Cell 3 — Split stratifié + scaling fit-on-train
# =========================
#  split AVANT le fit du scaler pour éviter la fuite d'information
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Standardisation (fit sur TRAIN uniquement, puis transform sur TRAIN & TEST)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

print(f"Taille du train set : {X_train.shape[0]} échantillons")
print(f"Taille du test set  : {X_test.shape[0]} échantillons")


Taille du train set : 398 échantillons
Taille du test set  : 171 échantillons


TensorDataset & DataLoaders

In [5]:
# =========================
# Cell 4 — TensorDataset & DataLoaders
# =========================
# Conversion en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor  = torch.tensor(X_test,  dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.long)

# Datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor,  y_test_tensor)

# DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

print("Batch size :", batch_size)


Batch size : 64


Device (CPU)

In [6]:
# =========================
# Cell 5 — Device (CPU fixé)
# =========================
import torch
device = torch.device("cpu")
print("Device utilisé :", device)


Device utilisé : cpu


# definition de model MLP

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size=30, hidden_sizes=[128, 64, 32], dropout_rate=0.5):
        super(MLP, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])

        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])

        self.fc4 = nn.Linear(hidden_sizes[2], 2)

        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)

        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)

        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)

        return self.fc4(x)



# entrainement de model

In [8]:
import torch.optim as optim
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/30, Loss: 0.6912
Epoch 2/30, Loss: 0.5687
Epoch 3/30, Loss: 0.4829
Epoch 4/30, Loss: 0.4133
Epoch 5/30, Loss: 0.3900
Epoch 6/30, Loss: 0.3306
Epoch 7/30, Loss: 0.3098
Epoch 8/30, Loss: 0.2743
Epoch 9/30, Loss: 0.2753
Epoch 10/30, Loss: 0.2231
Epoch 11/30, Loss: 0.2381
Epoch 12/30, Loss: 0.2644
Epoch 13/30, Loss: 0.1964
Epoch 14/30, Loss: 0.1897
Epoch 15/30, Loss: 0.2102
Epoch 16/30, Loss: 0.1418
Epoch 17/30, Loss: 0.1673
Epoch 18/30, Loss: 0.1482
Epoch 19/30, Loss: 0.1644
Epoch 20/30, Loss: 0.1791
Epoch 21/30, Loss: 0.1329
Epoch 22/30, Loss: 0.1191
Epoch 23/30, Loss: 0.1344
Epoch 24/30, Loss: 0.1231
Epoch 25/30, Loss: 0.1198
Epoch 26/30, Loss: 0.1311
Epoch 27/30, Loss: 0.1103
Epoch 28/30, Loss: 0.1756
Epoch 29/30, Loss: 0.0967
Epoch 30/30, Loss: 0.1270


# evaluation de model sur donnees propre et adv

In [9]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)

model.eval()
all_preds, all_targets, all_probs = [], [], []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)                            # logits
        probs = F.softmax(outputs, dim=1)[:, 1].cpu()      # proba classe positive (1)
        preds = torch.argmax(outputs, dim=1).cpu()         # prédiction (argmax)

        all_probs.extend(probs.numpy())
        all_preds.extend(preds.numpy())
        all_targets.extend(targets.numpy())                # targets sont déjà sur CPU

print("📊 Évaluation modèle standard (MLP) sur données propres :")
print(f"Accuracy       : {accuracy_score(all_targets, all_preds):.4f}")
print(f"Precision      : {precision_score(all_targets, all_preds, zero_division=0):.4f}")
print(f"Recall         : {recall_score(all_targets, all_preds, zero_division=0):.4f}")
print(f"F1-score       : {f1_score(all_targets, all_preds, zero_division=0):.4f}")
print(f"AUC-ROC        : {roc_auc_score(all_targets, all_probs):.4f}")


📊 Évaluation modèle standard (MLP) sur données propres :
Accuracy       : 0.9825
Precision      : 1.0000
Recall         : 0.9531
F1-score       : 0.9760
AUC-ROC        : 0.9985


In [10]:
!pip install adversarial-robustness-toolbox


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00


In [11]:
from art.estimators.classification import PyTorchClassifier
import torch.nn as nn
import torch.optim as optim

clip_min = X_train_tensor.min(dim=0).values.numpy()
clip_max = X_train_tensor.max(dim=0).values.numpy()

classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(30,),
    nb_classes=2,
    clip_values=(clip_min, clip_max),
    device_type="cpu"
)



#Préparation NumPy du set de test (standardisé) pour ART

In [12]:
import numpy as np

X_test_np = X_test_tensor.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()

model.eval()

MLP(
  (fc1): Linear(in_features=30, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (bn3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [13]:
model.eval()
def evaluate_attack(X_adv_np, y_true_np, attack_name):
    X_adv_tensor = torch.tensor(X_adv_np, dtype=torch.float32).to(device)
    with torch.no_grad():
        outputs = model(X_adv_tensor)
        probs = torch.softmax(outputs, dim=1)[:, 1].cpu().numpy()
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

    acc = accuracy_score(y_true_np, preds)
    precision = precision_score(y_true_np, preds, zero_division=0)
    recall = recall_score(y_true_np, preds, zero_division=0)
    f1 = f1_score(y_true_np, preds, zero_division=0)
    auc = roc_auc_score(y_true_np, probs)

    print(f"\n📉 📊 Résultats après attaque {attack_name}")
    print(f"Accuracy       : {acc:.4f}")
    print(f"Precision      : {precision:.4f}")
    print(f"Recall         : {recall:.4f}")
    print(f"F1-score       : {f1:.4f}")
    print(f"AUC-ROC        : {auc:.4f}")



In [14]:
from art.attacks.evasion import (
    FastGradientMethod,
    ProjectedGradientDescent,
    BasicIterativeMethod,
    CarliniL2Method
)

# FGSM
for eps in [0.1, 0.2, 0.3]:
    fgsm = FastGradientMethod(estimator=classifier, eps=eps)
    X_adv = fgsm.generate(x=X_test_np)
    evaluate_attack(X_adv, y_test_np, f"FGSM (eps={eps})")

# PGD
for eps in [0.1, 0.2, 0.3]:
    pgd = ProjectedGradientDescent(estimator=classifier, eps=eps, eps_step=eps/10,
                                   max_iter=20, norm=np.inf, targeted=False, num_random_init=0)
    X_adv = pgd.generate(x=X_test_np)
    evaluate_attack(X_adv, y_test_np, f"PGD (eps={eps}, step={eps/10}, it=20)")


# BIM
for eps in [0.1, 0.2, 0.3]:
    bim = BasicIterativeMethod(estimator=classifier, eps=eps, eps_step=eps/10, max_iter=10)
    X_adv = bim.generate(x=X_test_np)
    evaluate_attack(X_adv, y_test_np, f"BIM (eps={eps}, step={eps/10}, it=10)")

# C&W — config rapide
cw_fast = CarliniL2Method(classifier=classifier, confidence=0.0, targeted=False,
                          learning_rate=0.02, max_iter=75, binary_search_steps=1,
                          initial_const=0.3, batch_size=64)
X_cw_adv_fast = cw_fast.generate(x=X_test_np)
evaluate_attack(X_cw_adv_fast, y_test_np, "C&W-L2 (conf=0, c0=0.3, it=75, bsearch=1)")

# C&W — config forte
X_cw_input = X_test_np
y_cw_input = y_test_np
cw_hard = CarliniL2Method(classifier=classifier, confidence=0.0, targeted=False,
                          learning_rate=0.01, max_iter=500, binary_search_steps=7,
                          initial_const=0.01, batch_size=64)
X_cw_adv_hard = cw_hard.generate(x=X_cw_input)
evaluate_attack(X_cw_adv_hard, y_cw_input,
                "C&W-L2 STRONG (conf=0, c0=0.01, it=500, bsearch=7, lr=0.01)")



📉 📊 Résultats après attaque FGSM (eps=0.1)
Accuracy       : 0.9357
Precision      : 0.9818
Recall         : 0.8438
F1-score       : 0.9076
AUC-ROC        : 0.9942

📉 📊 Résultats après attaque FGSM (eps=0.2)
Accuracy       : 0.8713
Precision      : 0.8750
Recall         : 0.7656
F1-score       : 0.8167
AUC-ROC        : 0.9496

📉 📊 Résultats après attaque FGSM (eps=0.3)
Accuracy       : 0.7836
Precision      : 0.7368
Recall         : 0.6562
F1-score       : 0.6942
AUC-ROC        : 0.8332


PGD - Batches:   0%|          | 0/6 [00:00<?, ?it/s]


📉 📊 Résultats après attaque PGD (eps=0.1, step=0.01, it=20)
Accuracy       : 0.9357
Precision      : 0.9818
Recall         : 0.8438
F1-score       : 0.9076
AUC-ROC        : 0.9939


PGD - Batches:   0%|          | 0/6 [00:00<?, ?it/s]


📉 📊 Résultats après attaque PGD (eps=0.2, step=0.02, it=20)
Accuracy       : 0.8713
Precision      : 0.8750
Recall         : 0.7656
F1-score       : 0.8167
AUC-ROC        : 0.9406


...


📉 📊 Résultats après attaque BIM (eps=0.3, step=0.03, it=10)
Accuracy       : 0.7602
Precision      : 0.6949
Recall         : 0.6406
F1-score       : 0.6667
AUC-ROC        : 0.8172


C&W L_2:   0%|          | 0/3 [00:00<?, ?it/s]


📉 📊 Résultats après attaque C&W-L2 (conf=0, c0=0.3, it=75, bsearch=1)
Accuracy       : 0.9240
Precision      : 0.9474
Recall         : 0.8438
F1-score       : 0.8926
AUC-ROC        : 0.9947


C&W L_2:   0%|          | 0/3 [00:00<?, ?it/s]


📉 📊 Résultats après attaque C&W-L2 STRONG (conf=0, c0=0.01, it=500, bsearch=7, lr=0.01)
Accuracy       : 0.7953
Precision      : 0.7302
Recall         : 0.7188
F1-score       : 0.7244
AUC-ROC        : 0.9501
